# Análisis de Preferencias Musicales entre Springfield y Shelbyville

# 1. Introducción

Este proyecto tiene como propósito analizar las preferencias musicales de los usuarios de dos ciudades ficticias: Springfield y Shelbyville. A partir de un conjunto de datos de transmisión musical en línea, se busca comprobar si existen diferencias significativas en el comportamiento de los usuarios según el día de la semana y la ciudad en la que residen.

El análisis incluye tres etapas principales: revisión de los datos, preprocesamiento y prueba de hipótesis. Este enfoque simula el flujo de trabajo habitual de un analista de datos, quien debe validar supuestos a partir de evidencia empírica y comunicar los hallazgos de forma estructurada.

**Objetivo**

El objetivo principal es comprobar la siguiente hipótesis:

> **La actividad de los usuarios varía según el día de la semana y la ciudad.**

**Etapas**

Los datos utilizados se encuentran en el archivo `/datasets/music_project_en.csv`. No se dispone de información previa sobre la calidad del conjunto de datos, por lo que es necesario evaluarla antes de aplicar cualquier análisis.

El proyecto se divide en tres etapas:

1. Revisión inicial de los datos
2. Preprocesamiento y limpieza
3. Prueba de hipótesis







# 2. Revisión inicial de los datos

Se inicia el análisis cargando el archivo `music_project_en.csv` y revisando sus primeras filas y estructura general. Esto permite tener una visión preliminar sobre los datos disponibles y su formato.


In [82]:
# Importar librería y cargar el conjunto de datos
import pandas as pd

df = pd.read_csv(r"D:\Datasets\music_project_en.csv")

# Mostrar las primeras 10 filas del DataFrame
print(df.head(10))

# Mostrar información general del DataFrame
print(df.info())

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                       Chains          Obladaet  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE                     L’estate       Julia Dalia  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

        City        time        Day  
0  Shelbyville  20:28:33  Wednesday  
1  Springfield  14:07:09     Friday  
2  Shelbyville  20:58:07  Wednesday  
3  Shelbyville  08:37:09     Monday  
4  Springfield  08:34:34     Monday  
5

El conjunto de datos contiene 65,079 registros distribuidos en siete columnas. Todas las columnas están tipadas como `object`, por lo que se deberá evaluar si es necesario convertir algunas a otros formatos. A continuación se detallan las variables presentes:

- `userID`: identificador del usuario o la usuaria.
- `Track`: título de la canción.
- `artist`: nombre del artista.
- `genre`: género de la pista.
- `City`: ciudad en la que se reprodujo la canción.
- `time`: hora exacta de reproducción.
- `Day`: día de la semana.

**Posibles problemas detectados:**

- Algunos encabezados están en mayúsculas, otros en minúsculas.
- Hay espacios y formatos inconsistentes en los nombres de las columnas.
- Existen valores nulos en las columnas `Track`, `artist` y `genre`.
- Pueden existir registros duplicados.

Los datos parecen suficientes para el análisis, pero será necesario aplicar limpieza antes de realizar una prueba de hipótesis.


# 3. Preprocesamiento de datos

En esta sección se preparan los datos para su análisis. El primer paso consiste en corregir los encabezados, que presentan inconsistencias de formato. Posteriormente se tratarán los valores ausentes y duplicados.

**Limpieza de los nombres de columnas**

Al visualizar los encabezados, se detectan problemas como:
- Uso de mayúsculas y minúsculas inconsistentes.
- Espacios innecesarios.
- Nombres sin formato estandarizado (snake_case).

Se aplicarán transformaciones para garantizar que los nombres sean claros y uniformes.


In [86]:
# Mostrar los nombres originales de las columnas
print(df.columns)

# Convertir a minúsculas
df.columns = [col.lower() for col in df.columns]

# Eliminar espacios en blanco
df.columns = [col.strip() for col in df.columns]

# Corregir nombre específico a snake_case
df.rename(columns={'userid': 'user_id'}, inplace=True)

# Comprobar resultado final
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')
Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


Tras aplicar estas transformaciones, los encabezados quedan en formato limpio y estandarizado.  
Por ejemplo, `userID` fue renombrado correctamente como `user_id`, y todas las columnas ahora están en minúsculas y sin espacios.

Este formato facilitará el análisis posterior

**Valores ausentes**

A continuación se evalúan los valores ausentes en el conjunto de datos. Si bien no todos afectan directamente el análisis, es importante tratarlos para evitar errores en etapas posteriores.

Las columnas `track` y `artist` presentan valores faltantes que no afectan de forma crítica el análisis. Se reemplazarán con el valor `'unknown'` para mantener consistencia.

Sin embargo, los valores ausentes en `genre` podrían influir en los resultados, ya que esta columna está relacionada con las preferencias musicales. Por lo tanto, también será tratada de forma similar.

Se aplicará una estrategia de imputación simple reemplazando los valores ausentes con `'unknown'` en las tres columnas mencionadas.


In [90]:
# Calcular el número de valores ausentes antes del reemplazo
valores_ausentes = df.isnull().sum()
print("Valores ausentes antes del reemplazo:\n", valores_ausentes)

# Reemplazar valores ausentes por 'unknown'
columnas = ['track', 'artist', 'genre']
for col in columnas:
    df[col] = df[col].fillna('unknown')

# Verificar si aún existen valores ausentes
valores_ausentes_reemplazados = df.isnull().sum()
print("\nValores ausentes después del reemplazo:\n", valores_ausentes_reemplazados)


Valores ausentes antes del reemplazo:
 user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64

Valores ausentes después del reemplazo:
 user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


Luego del reemplazo, ya no quedan valores nulos en las columnas seleccionadas. Esto asegura que las futuras operaciones de agrupamiento y comparación no se vean afectadas por datos faltantes.

**Limpieza de duplicados**

A continuación se realiza la eliminación de duplicados tanto explícitos como implícitos en el conjunto de datos.

Los duplicados explícitos corresponden a filas completamente idénticas, mientras que los duplicados implícitos se manifiestan como errores de escritura en columnas clave, como `genre`.

Se comienza revisando la existencia de duplicados explícitos y eliminándolos del DataFrame.


In [94]:
# Contar y eliminar duplicados explícitos
duplicados = df.duplicated().sum()
print(f"Número de duplicados antes de la eliminación: {duplicados}")

df = df.drop_duplicates()

duplicados_2 = df.duplicated().sum()
print(f"Número de duplicados después de la eliminación: {duplicados_2}")


Número de duplicados antes de la eliminación: 3826
Número de duplicados después de la eliminación: 0


In [96]:
# Revisar los géneros únicos antes de la corrección
print("Valores únicos antes de la corrección:")
print(sorted(df['genre'].unique()))

# Definir función de reemplazo
def replace_wrong_genres(column, wrong_genres, correct_genre):
    for wrong in wrong_genres:
        column.replace(wrong, correct_genre, inplace=True)

# Reemplazar duplicados implícitos en 'genre'
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'

replace_wrong_genres(df['genre'], wrong_genres, correct_genre)

# Verificar cambios
print("\nValores únicos después de la corrección:")
print(sorted(df['genre'].unique()))


Valores únicos antes de la corrección:
['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans', 'alternative', 'ambient', 'americana', 'animated', 'anime', 'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook', 'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber', 'children', 'chill', 'chinese', 'choral', 'christian', 'christmas', 'classical', 'classicmetal', 'club', 'colombian', 'comedy', 'conjazz', 'contemporary', 'country', 'cuban', 'dance', 'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic', 'electropop', 'emo', 'entehno', 'epicmetal', 'estrada', 'ethnic', 'eurofolk', 'european', 'experimental', 'extrememetal', 'fado', 'film', 

Se eliminaron correctamente los duplicados explícitos del conjunto de datos.  
Además, se corrigieron los duplicados implícitos asociados al género musical "hiphop", consolidando las variantes `hip`, `hop` y `hip-hop` bajo una sola categoría uniforme.

# 4. Prueba de hipótesis

El objetivo de esta etapa es comprobar si existen diferencias en el comportamiento musical entre los usuarios de Springfield y Shelbyville, considerando tres días específicos: lunes, miércoles y viernes.

Para ello se agruparon los datos por ciudad y por día, comparando la cantidad de canciones reproducidas por cada grupo. Además, se implementó una función personalizada para contar las canciones reproducidas por ciudad y por día, facilitando la comparación entre ambas localidades.


In [101]:
# Agrupar por ciudad y contar la cantidad de canciones reproducidas
canciones_por_ciudad = df.groupby('city')['track'].count()
print(canciones_por_ciudad)

city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64


In [103]:
# Agrupar por día (lunes, miércoles, viernes) y ciudad
canciones_por_dia_y_ciudad = df[df['day'].isin(['Monday', 'Wednesday', 'Friday'])].groupby(['day', 'city'])['track'].count()
print(canciones_por_dia_y_ciudad)

day        city       
Friday     Shelbyville     5895
           Springfield    15945
Monday     Shelbyville     5614
           Springfield    15740
Wednesday  Shelbyville     7003
           Springfield    11056
Name: track, dtype: int64


In [107]:
def number_tracks(day, city):
    filtered = df[(df['day'] == day) & (df['city'] == city)]
    return filtered['user_id'].count()

In [109]:
# Resultados para cada ciudad y cada día
print("Springfield - Lunes:", number_tracks('Monday', 'Springfield'))
print("Shelbyville - Lunes:", number_tracks('Monday', 'Shelbyville'))
print("Springfield - Miércoles:", number_tracks('Wednesday', 'Springfield'))
print("Shelbyville - Miércoles:", number_tracks('Wednesday', 'Shelbyville'))
print("Springfield - Viernes:", number_tracks('Friday', 'Springfield'))
print("Shelbyville - Viernes:", number_tracks('Friday', 'Shelbyville'))

Springfield - Lunes: 15740
Shelbyville - Lunes: 5614
Springfield - Miércoles: 11056
Shelbyville - Miércoles: 7003
Springfield - Viernes: 15945
Shelbyville - Viernes: 5895


# 5. Conclusiones

Los resultados confirman que existen diferencias en el consumo de música entre las ciudades de Springfield y Shelbyville según el día de la semana. Por ejemplo, en algunos días una ciudad presenta significativamente mayor número de reproducciones que la otra, lo que sugiere que el comportamiento musical no es homogéneo entre ambas localidades.

Es importante tener en cuenta que estas conclusiones se basan en una muestra limitada y podrían no representar el comportamiento real de toda la población. En contextos reales, se requeriría un análisis estadístico más riguroso para validar estas hipótesis.

Este análisis exploratorio proporciona una base sólida para futuras investigaciones más avanzadas con técnicas estadísticas y muestras más amplias.
